In [1]:
# This is a notebook that would apply the following techniques 
# RollingMeanDeviation
# Support - resistance 
# Momentum
# Hammer and Hanging Man

In [2]:
# Note that for all analysis we would use prev day info (high, low, open, close)
# For evaluation, we would assume that the trades are made on Close (not prev day close)


In [4]:
# Writing this notebook with the idea of making this as modular as possible


In [13]:
import pandas as pd
from pyxirr import xirr

In [9]:
# Loading the dataset 
# To convert this into a function ideally
df = pd.read_csv('nifty.csv')
df.rename(columns={'CLOSE':'Close', 'HistoricalDate':'Date'}, inplace=True)
df = df.sort_values('Date')
df  = df.reset_index()
df['prevClose'] = df.Close.shift(1)

In [10]:
## Rolling Mean deviation analysis

In [11]:
df['moving_avg_30'] = df.prevClose.rolling(window=30).mean()
df['qty_traded'] = df.apply(lambda row: 1 if (row.moving_avg_30 - row.prevClose)/row.prevClose > 0.06 else 0 , axis=1)
df['qty_traded'] =df.apply(lambda row: -1 if (row.prevClose -row.moving_avg_30)/row.prevClose > 0.06 else row.qty_traded , axis=1)

In [15]:
df['trading_price'] = df.qty_traded * -1.0 * df.Close
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['Close']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-2, 'mktval']
xirr(df.Date, df.inflow)

0.13657459607610345

In [16]:
## Support Resistance

In [17]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

def pythag(pt1, pt2):
    a_sq = (pt2[0] - pt1[0]) ** 2
    b_sq = (pt2[1] - pt1[1]) ** 2
    return sqrt(a_sq + b_sq)

def regression_ceof(pts):
    X = np.array([pt[0] for pt in pts]).reshape(-1, 1)
    y = np.array([pt[1] for pt in pts])
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0], model.intercept_

def local_min_max(pts):
    local_min = []
    local_max = []
    prev_pts = [(0, pts[0]), (1, pts[1])]
    for i in range(1, len(pts) - 1):
        append_to = ''
        if pts[i-1] > pts[i] < pts[i+1]:
            append_to = 'min'
        elif pts[i-1] < pts[i] > pts[i+1]:
            append_to = 'max'
        if append_to:
            if local_min or local_max:
                prev_distance = pythag(prev_pts[0], prev_pts[1]) * 0.5
                curr_distance = pythag(prev_pts[1], (i, pts[i]))
                if curr_distance >= prev_distance:
                    prev_pts[0] = prev_pts[1]
                    prev_pts[1] = (i, pts[i])
                    if append_to == 'min':
                        local_min.append((i, pts[i]))
                    else:
                        local_max.append((i, pts[i]))
            else:
                prev_pts[0] = prev_pts[1]
                prev_pts[1] = (i, pts[i])
                if append_to == 'min':
                    local_min.append((i, pts[i]))
                else:
                    local_max.append((i, pts[i]))
    return local_min, local_max


In [18]:
def get_rolling_swing(row):
    series = df[df.Date<row.Date]['Close']
    if len(series)<200:
        return [None, None]
    series = series[-200:]
    
    series.index = np.arange(series.shape[0])

    month_diff = series.shape[0] // 30
    if month_diff == 0:
        month_diff = 1

    smooth = int(2 * month_diff + 3)

    pts = savgol_filter(series, smooth, 3)

    local_min, local_max = local_min_max(pts)
    
    
    if len(local_min):
        local_min_slope, local_min_int = regression_ceof(local_min)
        support = (local_min_slope * np.array(series.index)) + local_min_int
    else:
        support = [None,]
        
    if len(local_max):
        local_max_slope, local_max_int = regression_ceof(local_max)
        resistance = (local_max_slope * np.array(series.index)) + local_max_int
    else :
        resistance = [None,]

    return [support[-1],resistance[-1]]

In [19]:
df[['support', 'resistance']] = df.apply(lambda row:get_rolling_swing(row), axis=1, result_type='expand')

In [20]:
df['supported'] = df.apply(lambda row: 1 if row.prevClose<row.support else 0 , axis =1)
df['resisted'] = df.apply(lambda row: 1 if row.prevClose>row.resistance else 0, axis =1)

In [21]:
df['rolling_support'] = df.supported.rolling(10, min_periods=1).sum()
df['rolling_resistance'] = df.resisted.rolling(10, min_periods=1).sum()

In [22]:
df['qty_traded'] = df.apply(lambda row: 1 if row.rolling_support == 1.0 else 0 , axis=1)
df['qty_traded'] =df.apply(lambda row: -1 if row.rolling_resistance == 1.0 else row.qty_traded , axis=1)
df['trading_price'] = df.qty_traded * -1.0 * df.Close
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['Close']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-2, 'mktval']
xirr(df.Date, df.inflow)

0.11014445278093385

In [23]:
# Evaluate hammer trade

In [24]:
# We want to buy when there is hammer candlestick : 
# open - close is negative but open to close gap is <0.5 high to low gap 

In [27]:
df['prevClose'] = df.Close.shift(1)
df['prevOpen'] = df.OPEN.shift(1)
df['prevHigh'] = df.HIGH.shift(1)
df['prevLow'] = df.LOW.shift(1)

In [28]:
def hammer_trade(row):
    
    if row.prevOpen > row.prevClose : 
        if (row.prevOpen - row.prevClose)/ (row.prevHigh - row.prevLow) < 0.25:
            return 1
    if row.prevClose > row.prevOpen : 
        if (row.prevClose - row.prevOpen)/ (row.prevHigh - row.prevLow) < 0.25:
            return -1
    else : 
        return 0

In [31]:
df['qty_traded'] = df.apply(lambda row: hammer_trade(row) , axis=1)
df['trading_price'] = df.qty_traded * -1.0 * df.Close
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['Close']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-3, 'mktval']
df['inflow'] = df.inflow.fillna(0)
xirr(df.Date, df.inflow)

0.1043419900765296

In [36]:
## Momentum
df['cur_idx'] = df.index

In [43]:
def get_momentum(row):
    if row.cur_idx < 120:
        return 0
    lag_30_price = float(df.iloc[[row.cur_idx - 30]]['prevClose'])
    lag_90_price = float(df.iloc[[row.cur_idx - 60]]['prevClose'])
    lag_120_price = float(df.iloc[[row.cur_idx - 120]]['prevClose'])
    
    cur_price = row.prevClose
    
    if((cur_price> lag_30_price) & (cur_price > lag_90_price) & (cur_price > lag_120_price)):
        return 1
    
    if ((cur_price< lag_30_price) & (cur_price < lag_90_price) & (cur_price < lag_120_price)):
        return -1
    
    
    return 0


In [44]:
df['momentum_ind'] = df.apply(lambda row: get_momentum(row), axis=1)

In [47]:
df = df[df['Date']>'2016-01-01']
df['qty_traded'] = df['momentum_ind']
df['trading_price'] = df.qty_traded * -1.0 * df.Close
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['Close']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-2, 'mktval']
xirr(df.Date, df.inflow)

/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/3136267993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qty_traded'] = df['momentum_ind']
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/3136267993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trading_price'] = df.qty_traded * -1.0 * df.Close
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/3136267993.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

0.13345682206348888

In [48]:
# default model

In [49]:
df['qty_traded'] = 1
df['trading_price'] = df.qty_traded * -1.0 * df.Close
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['Close']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-2, 'mktval']
xirr(df.Date, df.inflow)

/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/2895903110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qty_traded'] = 1
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/2895903110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trading_price'] = df.qty_traded * -1.0 * df.Close
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_1034/2895903110.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

0.09191463726153357

In [ ]:
# todo : weighted 
"""
Assume that the person has a daily income. whenever one has to devote some amount
it would be based on the balance that is available 

When someone runs the model - they should get an indication of whether to invest/divest for the best value

The default weights that I am thinking of are : 

Hammer : highest 
Avg deviation : Med High
Regular : Med low
Momentum : Low

rolling xirr
weights are assigned based on 
"""

"""
How to prove that something is valuable : show year on year profits 
Goal : 
Not a single year with losses 
Net positive on the index

Introduce no sell mode
"""